# Bug in iter_index() in itertools doc recipies
* https://github.com/python/cpython/pull/102360
* https://docs.python.org/3.11/library/itertools.html#itertools-recipes
* https://github.com/more-itertools/more-itertools/pull/690

The function `iter_index()` silently suppresses ValueError exceptions rased by `iterable` iterator.

In [1]:
# The function - copied from:
# https://github.com/pochmann/cpython/blob/148bde644195408ec68cd145b779f42a214bfbb1/Doc/library/itertools.rst?plain=1#L881
# https://docs.python.org/3.11/library/itertools.html#itertools-recipes

import operator
from itertools import islice

def iter_index(iterable, value, start=0):
    "Return indices where a value occurs in a sequence or iterable."
    # iter_index('AABCADEAF', 'A') --> 0 1 4 7
    try:
        seq_index = iterable.index
    except AttributeError:
        # Slow path for general iterables
        it = islice(iterable, start, None)
        i = start - 1
        try:
            while True:
                yield (i := i + operator.indexOf(it, value) + 1)
        except ValueError:
            pass
    else:
        # Fast path for sequences
        i = start - 1
        try:
            while True:
                yield (i := seq_index(value, i+1))
        except ValueError:
            pass

## Demonstration of the bug

In [2]:
sequence = 'AABCADEAF'

def assert_no_value(iterable, forbidden_value):
    """Pass the iterable but raise ValueError if forbidden_value is found."""
    for item in iterable:
        if item == forbidden_value:
            raise ValueError(f'Value {forbidden_value!r} is not allowed.')
        yield item

In [3]:
# Example from the documentation:

list(iter_index(sequence, 'A'))

[0, 1, 4, 7]

In [4]:
# The same with our assert not engaging:

list(iter_index(assert_no_value(sequence, 'X'), 'A'))

[0, 1, 4, 7]

In [5]:
# Here we should get ValueError exception
# but it is being silently suppressed by iter_index()

list(iter_index(assert_no_value(sequence, 'B'), 'A'))

[0, 1]